In [9]:
import os
import sys

# Configure the environment
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = '/srv/spark'

# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

# Add the PySpark/py4j to the Python Path
sys.path.insert(0, os.path.join(SPARK_HOME, "python", "build"))
sys.path.insert(0, os.path.join(SPARK_HOME, "python"))

In [10]:
#sc.stop()
from pyspark import  SparkContext
sc = SparkContext( 'local[*]', 'pyspark')

In [11]:
import csv
from StringIO import StringIO
def split(line):
    reader = csv.reader(StringIO(line))
    return reader.next()

In [12]:
def parse(stringarr):
    return [float(stringarr[x]) for x in range(len(stringarr))]

In [188]:
def spaceBound(x,minBound,maxBound):
    count =0
    for agentNo in range(len(x)):
        for centroid in range(len(x[agentNo][0])):
            for value in range(len(x[agentNo][0][centroid])):
                if x[agentNo][0][centroid][value]>maxBound[value] or x[agentNo][0][centroid][value] < minBound[value]:
                    count +=1
                    x[agentNo][0][centroid][value] = (maxBound[value]-minBound[value])*random.random() + minBound[value]
                    
    print "number of reinits : "+ str(count)

In [189]:
from operator import add
def clusterDist(agent, dataPoint):
    '''
    having a data point, and an agent, this returns the distance of the data Point to the nearest centroid in the agent
    '''
    dataPoint = np.array(dataPoint)
    mindist = np.linalg.norm(agent[0]-dataPoint)
    for i in range(1,len(agent)):
        mindist = min(mindist,np.linalg.norm(agent[i]-dataPoint))
    return mindist

def evaluateFitness(positions, data):
    icdistance = data.flatMap(lambda dp: list([(i,clusterDist(positions.value[i],dp)) for i in range(len(positions.value))])) 
    fitnessTuple = icdistance.reduceByKey(add).values().collect()
    return np.array(fitnessTuple)

In [190]:
def evalMass(fitness):
    worstfit = max(fitness)
    bestfit = min(fitness)
    total = sum(fitness)
    mass = np.array([(fitness[x]-worstfit)/(bestfit-worstfit) for x in range(len(fitness))])
    mass/=sum(mass)
    return mass

In [191]:
from munkres import Munkres
import numpy as np
def genForce(agentPair, positions,G):
    agent1 = np.array(positions.value[agentPair[0][1]])
    agent2 = np.array(positions.value[agentPair[1][1]])
    mass1 = agentPair[0][0]
    mass2 = agentPair[1][0]
    distmat = [[0 for x in range(len(agent1))] for y in range(len(agent2))]
    for x in range(len(distmat)):
        for y in range(len(distmat[x])):
            distmat[x][y] = np.linalg.norm(agent1[x]-agent2[y])
    m = Munkres()
    clusterPairs = m.compute(distmat)
    forcesOnClusters = list()
    for clusterid1,clusterid2 in clusterPairs:
        cluster1 = np.array(agent1[clusterid1])
        cluster2 = np.array(agent2[clusterid2])
        force = G*mass1*mass2*(cluster2 - cluster1)/(np.linalg.norm(cluster2 - cluster1) + 10**-6)
        forcesOnClusters.append([(agentPair[0][1] , clusterid1),force])
    return forcesOnClusters

'''
gField, earlier output was <agent id, force>
a will now be of the form < [agent id, clusterid] ,force >
'''
def gField(mass,positions,G,rNorm, iterCount,maxIter):
    zipmass = zip(mass, [i for i in range(len(mass))])
    sortedmass = sorted(zipmass, key = lambda elem:elem[0])
    finalPer = 2
    kbest = round((finalPer+(1- float(iterCount)/maxIter)*(100-finalPer))*float(len(mass))/100)
    sortedAgents = sc.parallelize(sortedmass)
    sortedAgents.map(lambda pair: (pair[0]*random.random(),pair[1]))
    agentPairs = sortedAgents.cartesian(sc.parallelize(sortedAgents.take(int(kbest))))
    forces = agentPairs.flatMap(lambda pair: genForce(pair,positions,G))
    f = forces.reduceByKey(lambda a,b : [a[x]+b[x] for x in range(len(a))])
    f = f.collect()
    a = [(f[i][0],f[i][1]/(mass[i]+10**(-8))) for i in range(len(mass))]
    return a



In [192]:
def bucketize(data, agent):
    labels = [0 for i in range(len(data))]
    for i in range(len(data)):
        dp = np.array(data[i])
        mindist = sys.maxint/1.0
        currdist=0
        minind=-1
        for j in range(len(agent)):
            centroid = np.array(agent[j])
            currdist = np.linalg.norm(dp-centroid)
            if mindist> currdist:
                mindist = currdist
                minind = j
        labels[i]=minind
    return labels
            
                

def displayPlot(data, bestAgent):
    totaldata = data.collect()
    labels = bucketize(totaldata,bestAgent)
    pos = [list(t) for t in zip(*totaldata)]
    trace = go.Scatter3d(x = pos[0],
                       y = pos[1],
                         z = pos[2],
                       mode = 'markers',
                       marker= dict(color= labels,colorscale='Viridis',size=8,showscale=True))
    plot([trace],show_link=False)

In [193]:
'''
will return all clusters generated by kMeans solution in [k]x[d] array
'''
def kMeans(data,k):
    solution = [data[int(random.random()*len(data))] for x in range(k)]
    for x in range(20):
        buckets = [list() for x in range(k)] #k x arbit
        for point in data:
            mindist = sys.maxint
            mincentroid = -1
            for centroid in range(len(solution)):
                distance = np.linalg.norm(solution[centroid]-point)
                if distance <mindist :
                    mindist = distance
                    mincentroid = centroid
            buckets[mincentroid].append(point)
        for c in range(len(solution)):
            newc = np.array([0.0 for x in range(len(solution[c]))])
            for point in buckets[c]:
                newc+=point
            newc/=len(buckets[c])
            solution[c]=newc
    return solution

In [223]:
import numpy as np
import random as random
from math import exp
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot,plot
import plotly.plotly as py
import plotly.graph_objs as go


def GSA(dataFilePath, k,N,maxIter,visualize = False):
    if visualize:
        plotly.offline.init_notebook_mode()
    data = sc.textFile(dataFilePath).map(split)
    data = data.map(lambda dp:dp[:4]).map(parse)
    data.cache()
    nod = len(data.first())
    size = data.count()
    x=[[[[0.0 for j in range(nod)] for y in range(k)] , 0.0 , i] for i in range(N)]
    v=[[[0.0 for j in range(nod)] for y in range(k)] for i in range(N)]
    rNorm=2
    alpha=2
    fitness = [0.0 for i in range(N)]
    mass = [0 for i in range (N)]
    bestAgent = [[0 for i in range(nod)] for y in range(k)]
    bestFitness = sys.maxint/1.0
    avgFitnessList = []
    
    #set bounds of data in rangemin and rangemax
    minBound = np.array(data.reduce(lambda a,b:[min(a[i],b[i]) for i in range(len(b))]))
    print "minimum boundaries of data : " + str(minBound)
    maxBound = np.array(data.reduce(lambda a,b:[max(a[i],b[i]) for i in range(len(b))]))
    print "maximum boundaries of data : " + str(maxBound)
    
    #evaluate value of g0
    g0 = (np.min(maxBound-minBound))/(N**0.6)
    print "choosing g0 as " + str(g0)
    allPoints = np.array(data.collect())
    #randomly initialise agent populations x between maxBound and minBound
    for agentNo in range(len(x)):
        for centroid in range(len(x[agentNo][0])):
            #for value in range(len(x[agentNo][0][centroid])):
            x[agentNo][0][centroid] = allPoints[int(random.random()*len(allPoints))]
                #x[agentNo][0][centroid][value] = (maxBound[value]-minBound[value])*random.random() + minBound[value]
            
    #for i in range(10):
    #   x.append([kMeans(allPoints,len(x[0])),0.0,len(x)])
    
    for iterCount in range(maxIter):
        #TODO:write spaceBound function
        spaceBound(x,minBound,maxBound)
        positions = sc.broadcast([list(t) for t in zip(*x)][0])
        
        fitness = evaluateFitness(positions,data)
        #print "fitness of all agents: "
        #print fitness
        
        #evaluate average fitness, and globally best agent till now
        avgFitness = np.average(fitness)
        avgFitnessList.append(avgFitness)
        localMaxFitness = np.min(fitness)
        if localMaxFitness < bestFitness:
            bestFitness= localMaxFitness
            bestAgent = x[np.argmax(fitness)][0]
            
                
        print "best fitness yet is " + str(bestFitness) + " and average is " + str(avgFitness)
        
        mass = evalMass(fitness)
        #print "mass of all agents : " + str(mass)
        #add this fitness data to the agents
        for i in range(len(x)):
            x[i][1] = mass[i]
        
        G = g0 #*exp(-alpha*iterCount/maxIter)
        a = gField(mass,positions,G,rNorm, iterCount,maxIter)
        
        
        for i in range(len(a)):
            agent,cluster = a[i][0]
            for dim in range(len(v[agent][cluster])):
                v[agent][cluster][dim] = v[agent][cluster][dim]*random.random() + a[i][1][dim]
        #print "updated velocity : "
        #print v
        #print "average movement: " + str(np.average(v))
        
        #update positions
        for agent in range(len(x)):
            for centroid in range(len(x[agent][0])):
                for dim in range(len(x[agent][0][centroid])):
                    x[agent][0][centroid][dim] += v[agent][centroid][dim]
        #print "updated positions : "
        #for agent in range(len(x)):
        #    print x[agent][0]
        
    if visualize==True:
        displayPlot(data,bestAgent)
        
    
    return bestAgent

In [224]:
path = r'/Users/Jai/Documents/code/python-learning/datasets/iris.data'
bestsol = GSA(path,3,100,100,visualize=True)

minimum boundaries of data : [ 4.3  2.   1.   0.1]
maximum boundaries of data : [ 7.9  4.4  6.9  2.5]
choosing g0 as 0.151429762675
number of reinits : 0
best fitness yet is 111.867852787 and average is 178.309399507
number of reinits : 6
best fitness yet is 107.22725559 and average is 176.802502107
number of reinits : 6
best fitness yet is 102.767191469 and average is 174.144738472
number of reinits : 9
best fitness yet is 100.707642621 and average is 174.445270011
number of reinits : 12
best fitness yet is 100.707642621 and average is 167.34543032
number of reinits : 16
best fitness yet is 100.707642621 and average is 169.08768845
number of reinits : 20
best fitness yet is 100.707642621 and average is 165.946535795
number of reinits : 24
best fitness yet is 100.707642621 and average is 170.94531605
number of reinits : 24
best fitness yet is 100.707642621 and average is 164.465707323
number of reinits : 24
best fitness yet is 100.707642621 and average is 165.216959938
number of reinit